In [3]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import mysql.connector

db = mysql.connector.connect(host = "localhost",
                            username = "root",
                            password = "Papu@3012",
                            database = "ecomm")

cur = db.cursor()

In [4]:
import numpy as np

### Query 01 :

In [6]:
query1 = """ select distinct customer_city as cities from customers """

cur.execute(query1)

data1 = cur.fetchall()


df1 = pd.DataFrame(data1, columns = ["Cities"])
df1.head()

,Cities
0,franca
1,sao bernardo do campo
2,sao paulo
3,mogi das cruzes
4,campinas


### Query 02 :

In [8]:
query2 = """ select count(order_id) , year(order_purchase_timestamp) as order_year
from orders
where order_purchase_timestamp = 2017; """

cur.execute(query2)

data2 = cur.fetchall()


df2 = pd.DataFrame(data2, columns = ["Total_orders","year"])
df2

,Total_orders,year
0,45101,2017


### Query 03 :

In [10]:
query3 = """ select upper(products.product_category), round(sum(payments.payment_value),2) as total_sales
from products join order_items
on products.product_id = order_items.product_id
join payments
on order_items.order_id = payments.order_id
group by products.product_category; """

cur.execute(query3)

data3 = cur.fetchall()

df3 = pd.DataFrame(data3, columns = ["Category","Total_sales"])
df3.head()

,Category,Total_sales
0,PERFUMERY,506738.66
1,FURNITURE DECORATION,1430176.39
2,TELEPHONY,486882.05
3,BED TABLE BATH,1712553.67
4,AUTOMOTIVE,852294.33


### Query 04 :

In [12]:
query4 = """ select ((sum(case when payment_installments >= 1 then 1 else 0 end) * 100) / count(*)) as percentage_for_installments
from payments; """

cur.execute(query4)

data4 = cur.fetchall()

df4 = pd.DataFrame(data4, columns = ["percentage_for_installments"])
df4

,percentage_for_installments
0,99.9981


### Query 05 :

In [14]:
query5 = """ select customer_state, count(customer_id) as number_of_customers
from customers
group by customer_state """

cur.execute(query5)

data5 = cur.fetchall()


df5 = pd.DataFrame(data5, columns = ["customer_state", "number_of_customers"])
df5.head()

,customer_state,number_of_customers
0,SP,41746
1,SC,3637
2,MG,11635
3,PR,5045
4,RJ,12852


### Query 06 :

In [16]:
query6 = """ SELECT 
    YEAR(order_purchase_timestamp) AS order_year,
    MONTHNAME(order_purchase_timestamp) AS month,
    COUNT(order_id) AS total_orders
FROM
    orders
WHERE
    order_purchase_timestamp = 2018
GROUP BY MONTHNAME(order_purchase_timestamp)
ORDER BY MONTH(order_purchase_timestamp); """

cur.execute(query6)

data6 = cur.fetchall()

df6 = pd.DataFrame(data6, columns = ["order_year", "Month", "Total_orders"])
df6

,order_year,Month,Total_orders
0,2018,January,7269
1,2018,February,6728
2,2018,March,7211
3,2018,April,6939
4,2018,May,6873
5,2018,June,6167
6,2018,July,6292
7,2018,August,6512
8,2018,September,16
9,2018,October,4


### Query 07(1) :

In [18]:
query7 = """ SELECT 
    city,
    ROUND(AVG(order_count), 2) AS average_number_of_products_per_order
FROM
    (SELECT 
        orders.order_id,
            customers.customer_city AS city,
            COUNT(order_items.product_id) AS order_count
    FROM
        order_items
    JOIN orders ON order_items.order_id = orders.order_id
    JOIN customers ON orders.customer_id = customers.customer_id
    GROUP BY orders.order_id , customers.customer_city) subquery
GROUP BY city; """

cur.execute(query7)

data7 = cur.fetchall()

df7 = pd.DataFrame(data7, columns = ["City", "Average_number_of_products_per_order"])
df7.head()

,City,Average_number_of_products_per_order
0,anapolis,1.13
1,sao paulo,1.16
2,brasilia,1.13
3,belo horizonte,1.14
4,lagoa santa,1.12


### Query 07(2) :

In [20]:
query7_2 = """ with count_per_order as
(select orders.order_id, 
orders.customer_id, 
count(order_items.order_id) AS order_count
from orders join order_items
on orders.order_id = order_items.order_id
group by orders.order_id, orders.customer_id)

select customers.customer_city, round(avg(count_per_order.order_count),2) as average_number_of_products_per_order
from customers join count_per_order
on customers.customer_id = count_per_order.customer_id
group by customers.customer_city; """

cur.execute(query7_2)

data7_2 = cur.fetchall()

df7_2 = pd.DataFrame(data7_2, columns = ["City", "Average_number_of_products_per_order"])
df7_2.head()

,City,Average_number_of_products_per_order
0,sao paulo,1.16
1,sao jose dos campos,1.14
2,porto alegre,1.17
3,indaial,1.12
4,treze tilias,1.27


### Query 08 :

In [22]:
query8 = """ SELECT 
    UPPER(products.product_category) as category,
    round((SUM(payments.payment_value)  / (select sum(payments.payment_value) from payments)) * 100, 2) AS category_wise_total_sales
FROM
    products
        JOIN
    order_items ON products.product_id = order_items.product_id
        JOIN
    payments ON order_items.order_id = payments.order_id
GROUP BY category; """

cur.execute(query8)

data8 = cur.fetchall()

df8 = pd.DataFrame(data8, columns = ["category", "category_wise_total_sales_in_percentage"])
df8.head()

,category,category_wise_total_sales_in_percentage
0,PERFUMERY,3.17
1,FURNITURE DECORATION,8.93
2,TELEPHONY,3.04
3,BED TABLE BATH,10.70
4,AUTOMOTIVE,5.32


### Query 09 :

In [24]:
query9 = """ SELECT 
    products.product_category,
    COUNT(order_items.order_id) AS total_orders,
    ROUND(AVG(order_items.price), 2) AS average_price
FROM
    products
        JOIN
    order_items ON products.product_id = order_items.product_id
GROUP BY products.product_category """

cur.execute(query9)

data9 = cur.fetchall()

df9 = pd.DataFrame(data9, columns = ["Category", "Total Orders", "Average Price"])
df9.head()

,Category,Total Orders,Average Price
0,HEALTH BEAUTY,9670,130.16
1,sport leisure,8641,114.34
2,Cool Stuff,3796,167.36
3,computer accessories,7827,116.51
4,Watches present,5991,201.14


In [25]:
arr1 = df9["Total Orders"]
arr2 = df9["Average Price"]

a= np.corrcoef([arr1,arr2])
print("The correlation between product price and the number of times a product has been purchased is ", a[0][1])

The correlation between product price and the number of times a product has been purchased is  -0.10631514167157562


### Query 10 :

In [27]:
query10 = """ select *, dense_rank() over(order by total_value desc) as rank_no from
(SELECT 
    order_items.seller_id,
    COUNT(order_items.order_id) AS total_number_of_sales,
    ROUND(SUM(payments.payment_value), 2) AS total_value
FROM
    order_items
        JOIN
    payments ON order_items.order_id = payments.order_id
GROUP BY order_items.seller_id) as a """

cur.execute(query10)

data10 = cur.fetchall()

df10 = pd.DataFrame(data10, columns = ["Seller_id", "Total_Number_of_Orders", "Total_revenue","Rank"])
df10.head()

,Seller_id,Total_Number_of_Orders,Total_revenue,Rank
0,7c67e1448b00f6e969d365cea6b010ab,1452,507166.91,1
1,1025f0e2d44d7041d6cf58b6550e0bfa,1462,308222.04,2
2,4a3ca9315b744ce9f8e9374361493884,2133,301245.27,3
3,1f50f920176fa81dab994f9023523100,2008,290253.42,4
4,53243585a1d6dc2643021fd1853d8905,437,284903.08,5


### Query 11 :

In [29]:
query11 = """ select *, avg(bill) over(partition by customer_id order by OrderTime rows between 2 preceding and current row ) as mov_avg
from
(select orders.customer_id, orders.order_purchase_timestamp as OrderTime,
payments.payment_value as bill
from payments join orders
on payments.order_id = orders.order_id)  as a """

cur.execute(query11)

data11 = cur.fetchall()

df11 = pd.DataFrame(data11, columns = ["customer_id", "OrderTime", "bill","mov_avg"])
df11.head()

,customer_id,OrderTime,bill,mov_avg
0,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,114.74,114.739998
1,000161a058600d5901f007fab4c27140,2017-07-16 09:40:32,67.41,67.410004
2,0001fd6190edaaf884bcaf3d49edf079,2017-02-28 11:06:43,195.42,195.419998
3,0002414f95344307404f0ace7a26f1d5,2017-08-16 13:09:20,179.35,179.350006
4,000379cdec625522490c315e70c7a9fb,2018-04-02 13:42:17,107.01,107.010002


### Query 12 :

In [31]:
query12 = """ select years, Months, total_revenue,round(sum(total_revenue) over(order by years, Months),2) as cumilative_value from 
(select 
year(orders.order_purchase_timestamp) as years,
month(orders.order_purchase_timestamp) as Months,
-- monthname(orders.order_purchase_timestamp) as Months,
round(sum(payments.payment_value),2) as total_revenue
from orders join payments
on orders.order_id = payments.order_id
group by years,Months
order by years, Months asc) as a; """

cur.execute(query12)

data12 = cur.fetchall()

df12 = pd.DataFrame(data12, columns = ["years", "Months", "Total_revenue","cumulative_value"])
df12

,years,Months,Total_revenue,cumulative_value
0,2016,9,252.24,252.24
1,2016,10,59090.48,59342.72
2,2016,12,19.62,59362.34
3,2017,1,138488.04,197850.38
4,2017,2,291908.01,489758.39
5,2017,3,449863.60,939621.99
6,2017,4,417788.03,1357410.02
7,2017,5,592918.82,1950328.84
8,2017,6,511276.38,2461605.22
9,2017,7,592382.92,3053988.14


### Query 13 :

In [33]:
query13 = """ with a as(select 
year(orders.order_purchase_timestamp) as years,
round(sum(payments.payment_value),2) as total_revenue
from orders join payments
on orders.order_id = payments.order_id
group by years
order by years asc)

select *,round((((total_revenue - lag(total_revenue,1) over (order by years)) / lag(total_revenue,1) over (order by years) ) * 100 ),2) as yoy_perc_growth from a; """

cur.execute(query13)

data13 = cur.fetchall()

df13 = pd.DataFrame(data13, columns = ["years", "Total_revenue","YoY % Growth"])
df13

,years,Total_revenue,YoY % Growth
0,2016,59362.34,NaN
1,2017,7249746.73,12112.7
2,2018,8699763.05,20.0


### Query 14 :

In [67]:
query14 = """with a as (select customers.customer_id,
min(orders.order_purchase_timestamp) first_order
from customers join orders
on customers.customer_id = orders.customer_id
group by customers.customer_id),

b as (select a.customer_id, count(distinct orders.order_purchase_timestamp) next_order
from a join orders
on orders.customer_id = a.customer_id
and orders.order_purchase_timestamp > first_order
and orders.order_purchase_timestamp < 
date_add(first_order, interval 6 month)
group by a.customer_id) 

select 100 * (count( distinct a.customer_id)/ count(distinct b.customer_id)) as retention_rate
from a left join b 
on a.customer_id = b.customer_id ;"""

cur.execute(query14)
data14 = cur.fetchall()

df14 = pd.DataFrame(data14, columns = ["retention_rate"] )
df14

,retention_rate
0,None


# Query 15 :

In [76]:
query15 = """ select * from
(select orders.customer_id, 
year(orders.order_purchase_timestamp) as purchase_year,
round(sum(payments.payment_value),2) as total_bill,
dense_rank() over(partition by year(orders.order_purchase_timestamp) order by sum(payments.payment_value) desc ) as customer_ranking
from orders join payments
on orders.order_id = payments.order_id
-- where rankNo <= 3
group by purchase_year,orders.customer_id) as a
where customer_ranking <= 3; """

cur.execute(query15)
data15 = cur.fetchall()

df15 = pd.DataFrame(data15, columns = ["customer_id", "purchase_year","total_bill", "customer_ranking"] )
df15

,customer_id,purchase_year,total_bill,customer_ranking
0,a9dc96b027d1252bbac0a9b72d837fc6,2016,1423.55,1
1,1d34ed25963d5aae4cf3d7f3a4cda173,2016,1400.74,2
2,4a06381959b6670756de02e07b83815f,2016,1227.78,3
3,1617b1357756262bfa56ab541c47bc16,2017,13664.08,1
4,c6e2731c5b391845f6800c97401a43a9,2017,6929.31,2
5,3fd6777bbce08a352fddd04e4a7cc8f6,2017,6726.66,3
6,ec5b2ba62e574342386871631fafd3fc,2018,7274.88,1
7,f48d464a0baaea338cb25f816991ab1f,2018,6922.21,2
8,e0a2412720e9ea4f26c1ac985f6a7358,2018,4809.44,3
